In [2]:
"""read in HNO data, convert ISOz"""

import pandas as pd
import country_converter as coco

def pull():
    LOC = '/Users/ewanog/Google Drive/SoHSS/Report/Docs/Datasets/hno_hrp_deep_export.xlsx'
    hno = pd.read_excel(LOC, sheet_name='Grouped Entries')

pull()